In [ ]:
!pip install transformers datasets evaluate
!pip install accelerate -U
!pip install transformers[torch]

필요 패키지 설치

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Huggingface 업로드
https://huggingface.co/BLACKBUN/Tesla_related_classification

In [2]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/combined_dict_list.json', 'r') as f:
		data = json.load(f)

print(data[:5])

[{'label': 1, 'text': "GM reaches deal for access to Tesla's North American chargers General Motors vehicles will have access to Tesla's charging network following an announcement by the two companies. Photo: JUSTIN SULLIVAN / GETTY IMAGES NORTH AMERICA/Getty Images via AFP/File Tesla will open its North American electric vehicle charging ..."}, {'label': 0, 'text': "Said Benrahma's strike sees Hammers through Manager David Moyes hopes West Ham United's FA Cup win at Brentford will prove to be a turning point in their season."}, {'label': 0, 'text': 'Sanna Marin fights for survival Polls suggest she is in a tight race with centre-right Petteri Orpo and right-wing populist Riikka Purra.'}, {'label': 1, 'text': 'Tesla shares finally snap record 13-day win streak that added $200 billion in value (Reuters) – Tesla’s record 13-day streak of gains finally came to an end on Wednesday, as the stock closed down just a smidgen after a stretch that saw the U.S. automaker add more than $200 billio

In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

print(f"Number of elements in training set: {len(train_data)}")
print(f"Number of elements in testing set: {len(test_data)}")

Number of elements in training set: 34775
Number of elements in testing set: 3864


데이터셋 학습:테스트 = 9:1 설정

In [4]:
from datasets import Dataset, DatasetDict

# Convert the data into HuggingFace dataset
hf_train = Dataset.from_dict({k: [d[k] for d in train_data] for k in train_data[0]})
hf_test = Dataset.from_dict({k: [d[k] for d in test_data] for k in test_data[0]})

# Print the first five rows
print(hf_train[:5])
print(hf_test[:5])


{'label': [0, 0, 0, 0, 1], 'text': ["Wolves sack manager after defeat by West Ham Bruno Lage is sacked as manager of Wolves after 16 months in charge following Saturday's defeat at West Ham United.", 'Schedule to be announced on Thursday The 2023-24 Premier League fixtures will be released at 09:00 BST on Thursday - find out how to follow your favourite team on BBC Sport.', 'Seriously injured playing rugby league for Ireland and left to cover the costs Charlie Willett suffered a serious knee injury playing rugby league for Ireland, but says she found out after she did not have full medical cover.', 'US man charged after threat to fly plane into Walmart The Mississippi shop was evacuated as police spoke to the pilot, who went on to land in a field.', 'Tesla’s Record Run Drives Nearly $200 Billion Jump in Value In fact, ardent Tesla bull and the chief executive officer of ARK Investment Management Cathie Wood has said that the company is the biggest AI play, which will help its stock pri

In [5]:
news_data = DatasetDict({
    'train': hf_train,
    'test': hf_test
})

Transformers 이용 위해 데이터 Dataset Class로 변환

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
def preprocess_function(examples):
		return tokenizer(examples["text"], truncation=True)

In [8]:
tokenized_news_data = news_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/34775 [00:00<?, ? examples/s]

Map:   0%|          | 0/3864 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

텍스트 토크나이징

In [10]:
import evaluate

accuracy = evaluate.load("accuracy")

In [11]:
import numpy as np

def compute_metrics(eval_pred):
		logits, labels = eval_pred
		predictions = np.argmax(logits, axis=1)
		return accuracy.compute(predictions=predictions, references=labels)

In [12]:
id2label = {0: "Unrelated", 1: "Related"}
label2id = {"Unrelated": 0, "Related": 1}

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2,
                                                           id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.

학습 전 필요 요소 정의

In [14]:
training_args = TrainingArguments(
    output_dir="Tesla_related_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news_data["train"],
    eval_dataset=tokenized_news_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/BLACKBUN/Tesla_related_classification into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.002800,0.004338,0.999224
2,0.000000,0.004195,0.999482


TrainOutput(global_step=4348, training_loss=0.004150733014563546, metrics={'train_runtime': 606.277, 'train_samples_per_second': 114.717, 'train_steps_per_second': 7.172, 'total_flos': 1456216183228512.0, 'train_loss': 0.004150733014563546, 'epoch': 2.0})

학습 및 결과 도출, 데이터 테스트셋 기준 정확도 99.9%

In [15]:
text = "Tesla Model S &amp; Model X have hospital-grade HEPA air filters, protecting you from dust, bacteria, pollen (allergies), spores &amp; many toxic gases"

In [16]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model='BLACKBUN/Tesla_related_classification')
classifier(text)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'Related', 'score': 0.9999575614929199}]